1. Secret key encryption - attempt 1
2. Secret key ecnryption - attempt 2
    Show additive homomorphism
    Show failure of additive homomorphism
    Show plaintext inner product
3. Show plaintext PIR
4. Show encrypted PIR

In [180]:
import random

class Matrix:

    def __init__(self, mod, rows, cols, mat):
        self.mod = mod
        self.rows = rows
        self.cols = cols
        self.mat = mat
        
    def scale(self, numerator, denominator, new_q):
        mat = [0] * (self.rows * self.cols)
        for index in range(len(self.mat)):
            mat[index] = round((numerator * self.mat[index])/denominator) % new_q
        return Matrix(new_q, self.rows, self.cols, mat)

                

    def __mul__(self, other):
        assert (self.cols == other.rows)
        mat = [0] * (self.rows * other.cols)
        for i in range(self.rows):
            for j in range(self.cols):
                for k in range(other.cols):
                    mat[i*other.cols + k] = (self.mat[i*self.cols + j] * other.mat[j*other.cols + k]) % self.mod
        return Matrix(self.mod, self.rows, other.cols, mat)
        

    def __add__(self, other):
        assert (self.rows == other.rows and self.cols == other.cols)
        for index in range(len(self.mat)):
            self.mat[index] = (self.mat[index] + other.mat[index]) % self.mod
        return self

    def __sub__(self, other):
        assert (self.rows == other.rows and self.cols == other.cols)
        for index in range(len(self.mat)):
            self.mat[index] = (self.mat[index] - other.mat[index]) % self.mod
        return self
    
    def __eq__(self, other):
        return self.rows == other.rows and self.cols == other.cols and self.mat == other.mat

    def __repr__(self):
        return "Rows:"+ str(self.rows) + " Cols:" + str(self.cols) + " \nMatrix(" + str(self.mat) + ")"

def random_matrix(mod, rows, cols): 
    mat = [random.randint(0, mod - 1) for _ in range(rows * cols)]
    return Matrix(mod, rows, cols, mat)

def sample_error(bound):
    return random.randint(-bound, +bound)

def sample_error_matrix(bound, mod, rows, cols): 
    mat = [sample_error(bound) % mod for _ in range(cols * rows)]
    return Matrix(mod, rows, cols, mat)

In [181]:
a = random_matrix(32, 10,10)
b = random_matrix(32, 10,10)

In [182]:
c = a + b
print(c)

Rows:10 Cols:10 
Matrix([7, 9, 28, 25, 17, 18, 10, 1, 6, 9, 0, 25, 30, 17, 7, 2, 10, 9, 21, 16, 7, 31, 24, 4, 11, 7, 27, 17, 20, 22, 22, 21, 25, 21, 2, 28, 19, 14, 16, 19, 7, 6, 5, 29, 25, 17, 3, 10, 24, 27, 17, 29, 5, 23, 19, 23, 26, 24, 2, 17, 0, 8, 25, 22, 3, 0, 18, 14, 8, 20, 18, 6, 16, 3, 21, 6, 24, 11, 5, 4, 5, 6, 6, 9, 13, 10, 6, 6, 18, 8, 17, 15, 11, 1, 12, 14, 3, 17, 21, 14])


Secret key regev ecnryption scheme

We paramterise the scheme with n, m, q, p. 

We have chosen parameter values for covenience of the tutorial and they are not secure. To find out more how to figure out secure values check here (INSERT LINK)

## Attempt 1

In [183]:
n = 10
m = 100
p = 50
bound = 2
q = 1000

e1 = sample_error(bound)

In [184]:
s = random_matrix(q, n, 1)
m0 = random_matrix(q, m, 1)

Encrypt 

In [185]:
A = random_matrix(q, m, n)
e = sample_error_matrix(bound, q, m, 1)
B = (A * s) + e + m0 
c = (B, A)

Decrypt

In [186]:
# Given c and secret s
B = c[0]
A = c[1]
m1 = B - (A * s)

# Notice that m1 != m0 
assert(m1 != m0)

# Since m1 = m0 + e, m1 - e == m0 
# TODO implement equality
assert((m1 - e) == m0)

## Attempt 2

Encrypt

In [187]:
delta = q / p
m0 = random_matrix(p, m, 1) 
m12 = m0
m0_scaled = m0.scale(q, p, q)
A = random_matrix(q, m, n)
e = sample_error_matrix(bound, q, m, 1)
B = (A * s) + e + m0_scaled 
c = (B, A)

Decrypt

In [188]:
B = c[0]
A = c[1]
m1 = B - (A * s)
m0_r = m1.scale(p, q, p)


In [189]:
print(m0_r)
print(m12)


Rows:100 Cols:1 
Matrix([3.05, 5.05, 18.95, 15.9, 4.0, 31.95, 35.9, 4.1, 46.05, 9.05, 33.95, 48.1, 32.9, 0.9, 18.0, 43.1, 17.05, 6.9, 34.05, 9.9, 34.05, 12.1, 32.1, 21.1, 46.1, 23.0, 42.0, 20.95, 12.0, 46.95, 15.9, 1.1, 15.9, 46.1, 33.95, 19.95, 46.1, 4.0, 26.05, 27.0, 48.1, 34.05, 25.1, 12.9, 49.05, 1.1, 47.9, 5.95, 26.9, 47.9, 42.0, 31.95, 36.1, 32.05, 10.95, 20.9, 47.1, 36.1, 35.05, 6.9, 3.9, 33.0, 11.1, 4.9, 19.95, 31.95, 35.1, 49.0, 46.1, 37.95, 44.0, 46.1, 14.1, 12.1, 10.1, 16.0, 24.0, 6.0, 13.9, 30.05, 5.1, 40.95, 12.9, 0.1, 18.0, 46.95, 5.05, 11.05, 2.9, 43.1, 46.05, 20.1, 48.05, 19.9, 28.0, 9.95, 13.95, 31.9, 4.9, 13.9])
Rows:100 Cols:1 
Matrix([3, 5, 19, 16, 4, 32, 36, 4, 46, 9, 34, 48, 33, 1, 18, 43, 17, 7, 34, 10, 34, 12, 32, 21, 46, 23, 42, 21, 12, 47, 16, 1, 16, 46, 34, 20, 46, 4, 26, 27, 48, 34, 25, 13, 49, 1, 48, 6, 27, 48, 42, 32, 36, 32, 11, 21, 47, 36, 35, 7, 4, 33, 11, 5, 20, 32, 35, 49, 46, 38, 44, 46, 14, 12, 10, 16, 24, 6, 14, 30, 5, 41, 13, 0, 18, 47, 5, 11, 3, 